In [1]:
import yfinance as yf
import pandas as pd
import requests
from yahooquery import Ticker
from datetime import datetime, timedelta
import os
from IPython.display import clear_output

In [2]:
#How do i #define in pyhton?
DAYS_IN_YEAR = 365
FIVE_YEARS_IN_DAYS = 5 *DAYS_IN_YEAR
end_date = pd.Timestamp.utcnow()
start_date = end_date - timedelta(days = FIVE_YEARS_IN_DAYS)

In [3]:
def get_historical_stock_data(ticker):
    tick = yf.Ticker(ticker)
    stock_data = tick.history(period='5y')
    return stock_data


In [8]:
etf_names = ['ICF', 'IYR', 'REET', 'REM', 'REZ', 'USRT']
#etf_names = ['ICF', 'IYR']


ticker_names = set()


cols = ['Ticker', 'ETF', 'Number of ETFs', 'Closing Stock Price']

#Data frame to hold all data
#Ticker, ETF's its in, Number of ETF's its in, stock price
df = pd.DataFrame(columns = cols)

dfs = []

for i, etf_name in enumerate(etf_names):
    dfs.append(pd.read_excel(f'Data\\{etf_name}.xlsx', skiprows=7, header=0))
    
    #Extracting list of stocks to pull prices
    for ticker_name in dfs[i]['Ticker'].to_numpy():
        ticker_names.add(ticker_name)

#ticker name, ETF's its in
tick_etf_list = {}

for ticker_name in ticker_names:
    etf_list = []
    for j, etf_name in enumerate(etf_names):
        if ticker_name in dfs[j]['Ticker'].to_list():
            etf_list.append(etf_name)
            
    tick_etf_list[ticker_name] = etf_list

#Should be at the end
for i, (ticker, etf_list) in enumerate(tick_etf_list.items()):
    
    #Using the Closing price for each stock
    print(f'Getting stock prices for {ticker}... {round(i / ticker_names.__len__(), 2) * 100} percent') 
    
    close_prices = get_historical_stock_data(ticker)['Close'].to_numpy()
    
    #Getting an error 'No data found, symbol may be delisted'
    #This is a known bug?
    #Just jump over the stocks that have this issue
    if close_prices.__len__() == 0:
        clear_output(wait=True)
        continue

    #Ignoring all close prices that do not have the correct size (1258)
    #This turns out to be ~22.4% of stocks we are just dropping, so
    #in the future we might want to fix this (interpolation?)
    if close_prices.__len__() < 1258:
        clear_output(wait=True)
        continue


    #With these two checks in place, we are left with ~40% of our original stock count
    number_of_etfs = len(etf_list)
    row = {'Ticker': ticker, 'ETF': etf_list, 'Number of ETFs': number_of_etfs, 'Closing Stock Price': close_prices}
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    clear_output(wait=True)

Percentage_left = df['Ticker'].to_numpy().__len__() / ticker_names.__len__()
print(f'Percentage of stocks left after normalization: {round(Percentage_left, 4)* 100}%')

Percentage of stocks left after normalization: 39.81%


In [5]:
deficient_count = 0
for tmp in df['Closing Stock Price'].to_numpy():
    if tmp.__len__() / 1258 != 1.0:
        deficient_count+=1
print(deficient_count/ len(df['Closing Stock Price'].to_numpy()))

0.0


In [9]:
df.to_excel('Normalized Data.xlsx', index=False)

0.3980815347721823

In [22]:
temp = yf.Ticker("URW")
temp.history(start='2022-01-01', end='2023-01-01')

URW: No price data found, symbol may be delisted (1d 2022-01-01 -> 2023-01-01)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
